In [1]:
%env PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0

env: PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0


In [2]:
# %% Imports
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import copy
import pandas as pd 
import os

In [3]:
device = torch.device('mps')

In [4]:
TRAIN_ROOT = "data/brain_mri/training"
TEST_ROOT = "data/brain_mri/testing"

In [5]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.resnet50 = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        num_ftrs = self.resnet50.fc.in_features
        self.resnet50.fc = nn.Linear(num_ftrs, 4)

    def forward(self, x):
        x = self.resnet50(x)
        return x

In [6]:
model = CNNModel()
model = model.to(device)

In [7]:
model

CNNModel(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
        

In [5]:
train_dataset = torchvision.datasets.ImageFolder(
        root=TRAIN_ROOT,
        transform=transforms.Compose([
                      transforms.Resize((255,255)),
                      transforms.ToTensor()
        ])
)

test_dataset = torchvision.datasets.ImageFolder(
        root=TEST_ROOT,
        transform=transforms.Compose([
                      transforms.Resize((255,255)),
                      transforms.ToTensor()
        ])
)

In [6]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True
)


In [10]:
cross_entropy_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
epochs = 5

In [30]:
for epoch in range(epochs):  
    for i, batch in enumerate(train_loader, 0):
        inputs, labels = batch
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        # Labels are automatically one-hot-encoded
        loss = cross_entropy_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        print("This is loss-->",loss)
    print("new epoch started")

This is loss--> tensor(1.3697, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3748, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3691, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3429, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3807, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3712, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3602, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3816, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3607, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3494, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3546, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3758, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.3366, device='mps:0', grad_fn=<NllLossBackward0>)
This is loss--> tensor(1.

In [ ]:
import pandas as pd
inputs, labels = next(iter(test_loader))
inputs = inputs.to(device)
labels = labels.numpy()
outputs = model(inputs).max(1).indices.detach().cpu().numpy()
comparison = pd.DataFrame()
print("Batch accuracy: ", (labels==outputs).sum()/len(labels))
comparison["labels"] = labels

comparison["outputs"] = outputs
comparison

In [7]:
nmodel = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
nmodel = nmodel.to(device)

# Modify the last fully connected layer to fit the number of classes in the brain MRI dataset
num_classes = 4 # assuming 2 classes: tumor and non-tumor
in_features = nmodel.fc.in_features
nmodel.fc = nn.Linear(in_features, num_classes)


In [8]:
nmodel.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(nmodel.parameters(), lr=0.001)

In [10]:

num_epochs = 5
for epoch in range(num_epochs):
    # Train for one epoch
    running_loss = 0.0
    vgg16.train()
    for inputs, labels in train_loader:
        # Forward pass
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = nmodel(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        print("the running losss is ---->", loss)
    epoch_loss = running_loss / len(train_loader.dataset)
    print("epoch loss---->",epoch_loss)

the running losss is ----> tensor(1.3809, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(1.1054, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(1.1277, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.8072, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.6591, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.5199, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(1.0155, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.6345, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.9736, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.6754, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.5082, device='mps:0', grad_fn=<NllLossBackward0>)
the running losss is ----> tensor(0.6273, device='mps:

In [12]:
import pandas as pd
inputs, labels = next(iter(test_loader))
inputs = inputs.to(device)
labels = labels.numpy()
outputs = nmodel(inputs).max(1).indices.detach().cpu().numpy()
comparison = pd.DataFrame()
print("Batch accuracy: ", (labels==outputs).sum()/len(labels))
comparison["labels"] = labels

comparison["outputs"] = outputs
comparison

Batch accuracy:  0.84375


,labels,outputs
0,1,1
1,3,3
2,0,0
3,2,2
4,3,3
5,1,1
6,2,2
7,0,0
8,1,1
9,2,2


In [154]:
!pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 kB/s eta 0:00:0000:0100:02m


In [176]:
images, labels = next(iter(test_loader))
image = images[30]
label = labels[30]
image = image.to(device)
label = label.to(device)
label
image = image.unsqueeze(0)
image.shape


torch.Size([1, 3, 255, 255])

In [179]:
for param in nmodel.parameters():
    param.requires_grad = False
    

In [193]:
def forward_hook(module, input, output):
    module.relevance = output.detach()

for name, module in nmodel.named_modules():
    module.register_forward_hook(forward_hook)
    
def lrp(model, input, output):
    relevance = output
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            w = module.weight
            b = module.bias
            z = module.relevance
            a = module.output
            s = relevance / (a + 1e-9)
            c = (s @ w) + b
            relevance = z * c
        elif isinstance(module, torch.nn.Conv2d):
            w = module.weight
            b = module.bias
            z = module.relevance
            a = module.input[0]
            s = relevance / (a + 1e-9)
            c = torch.nn.functional.conv2d(z, w, stride=module.stride, padding=module.padding)
            c = c + b.view(1, -1, 1, 1)
            relevance = z * c
        elif isinstance(module, torch.nn.ReLU):
            z = module.input[0]
            a = module.output
            s = relevance / (a + 1e-9)
            relevance = z * (s * (a > 0).float())
        elif isinstance(module, torch.nn.MaxPool2d):
            z = module.input[0]
            a = module.output
            s = relevance / (a + 1e-9)
            relevance = z * torch.nn.functional.max_pool2d(s, kernel_size=module.kernel_size, stride=module.stride, padding=module.padding)
    return relevance


In [194]:
import matplotlib.pyplot as plt

In [195]:
output = nmodel(image)

relevance = lrp(nmodel, image, output)

heatmap = relevance.sum(dim=1, keepdim=True).squeeze()

plt.imshow(transforms.ToPILImage()(image.squeeze()), cmap='gray')
plt.imshow(heatmap.detach().numpy(), cmap='jet', alpha=0)

AttributeError: 'Conv2d' object has no attribute 'input'

In [203]:
!pip install install innvestigate

  Using cached Keras-2.2.4-py2.py3-none-any.whl (312 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-macos 2.12.0 requires keras<2.13,>=2.12.0, but you have keras 2.2.4 which is incompatible.


In [207]:
torch.save(nmodel, 'pretrained_model.pt')

PicklingError: Can't pickle <function forward_hook at 0x4547e2a60>: it's not the same object as __main__.forward_hook

In [201]:
import keras
print(keras.__version__)

2.12.0


In [200]:
!pip install keras --upgrade

  Using cached keras-2.12.0-py2.py3-none-any.whl (1.7 MB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.2.4
    Uninstalling Keras-2.2.4:
      Successfully uninstalled Keras-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
innvestigate 1.0.9 requires keras==2.2.4, but you have keras 2.12.0 which is incompatible.
